# Data Setup

In [1]:
#pip install protobuf==3.20.*

In [2]:
import sys
sys.path.append('helpermodule')

from helpermodule import data
from data import get_chapter_data, get_excerpt_data
from sklearn.model_selection import train_test_split

In [3]:
help(data)

Help on module helpermodule.data in helpermodule:

NAME
    helpermodule.data

FUNCTIONS
    get_chapter_data()
    
    get_excerpt_data(n_words: int = 100)
        #n_words: # words in an excerpt
    
    import_data()

FILE
    /home/vincentgu/DATASCI266-Final-Project-Gu/helpermodule/data.py




In [4]:
chapter_labels, chapter_examples = get_chapter_data()
print(chapter_labels[0])
print(chapter_examples[0][:199])

Virginia
“Citizens of the Solar Republic, this is your Sovereign.” I stare half blind into a ﬁring squad of ﬂy-eyed cameras. Out the viewport behind my stage, battle stations and ships of war ﬂoat beyond the 


In [5]:
excerpt_labels, excerpt_examples = get_excerpt_data(n_words=100)
print(excerpt_labels[100])
print(excerpt_examples[100])

1
mottled with the resFlesh that has replaced the chunks Atlas took out. New metal ﬁngers extend from her knuckles. “Trouble?” she asks. “Pushy relations.” Without a smile, she turns back to watch the polar sky. Beyond the atmosphere of the planet, Atalantia’s warships rove, waiting for us to just nip our heads outside the great shield chains so they can drop mass drivers down and make craters of us. “Cold back here,” I say over the whistling wind. Our ship passes over the edge of an ice shelf. “Why don’t you head to mess? Colloway says it’s bad to sync


In [6]:
x_train, x_test, y_train, y_test = train_test_split(excerpt_examples, excerpt_labels, test_size=.2, random_state=2457)
print(len(x_train))
print(len(x_test))
print(len(y_train))
print(len(y_test))

2257
565
2257
565


# BERT Setup

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K



import sklearn as sk
import os
import nltk
from nltk.data import find

import matplotlib.pyplot as plt

import re

2023-06-03 21:34:57.106511: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
from transformers import BertTokenizer, TFBertModel

from transformers import logging
logging.set_verbosity_error()

In [9]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
bert_model = TFBertModel.from_pretrained('bert-base-cased')

2023-06-03 21:35:53.534504: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Basic BERT Text Classification

In [10]:
x_train_tf = tf.convert_to_tensor(x_train)
x_test_tf = tf.convert_to_tensor(x_test)
y_train_tf = tf.convert_to_tensor(y_train)
y_test_tf = tf.convert_to_tensor(y_test)

In [11]:
num_train_examples = 2000      # set number of train examples
num_test_examples = 500        # set number of test examples

MAX_SEQUENCE_LENGTH = 128                 # set max_length of the input sequence

all_train_examples = [x.decode('utf-8') for x in x_train_tf.numpy()]
all_test_examples = [x.decode('utf-8') for x in x_test_tf.numpy()]

x_train_tf = bert_tokenizer(all_train_examples[:num_train_examples],
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
y_train_tf = y_train_tf[:num_train_examples]

x_test_tf = bert_tokenizer(all_test_examples[:num_test_examples],
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
y_test_tf = y_test_tf[:num_test_examples]

In [12]:
def create_bert_classification_model(bert_model,
                                     num_train_layers=0,
                                     hidden_size = 200, 
                                     dropout=0.3,
                                     learning_rate=0.00005):
    """
    Build a simple classification model with BERT. Use the Pooler Output for classification purposes
    """
    if num_train_layers == 0:
        # Freeze all layers of pre-trained BERT model
        bert_model.trainable = False

    elif num_train_layers == 12: 
        # Train all layers of the BERT model
        bert_model.trainable = True

    else:
        # Restrict training to the num_train_layers outer transformer layers
        retrain_layers = []

        for retrain_layer_number in range(num_train_layers):

            layer_code = '_' + str(11 - retrain_layer_number)
            retrain_layers.append(layer_code)
          
        
        print('retrain layers: ', retrain_layers)

        for w in bert_model.weights:
            if not any([x in w.name for x in retrain_layers]):
                #print('freezing: ', w)
                w._trainable = False

    input_ids = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}      

    bert_out = bert_model(bert_inputs)

    pooler_token = bert_out[1]
    #cls_token = bert_out[0][:, 0, :]

    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(pooler_token)


    hidden = tf.keras.layers.Dropout(dropout)(hidden)  


    classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden)
    
    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])
    
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), 
                                 metrics='accuracy')
    
    return classification_model

In [13]:
bert_classification_model = create_bert_classification_model(bert_model, num_train_layers=12)
#confirm all layers are frozen
bert_classification_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask_layer (InputLay  [(None, 128)]       0           []                               
 er)                                                                                              
                                                                                                  
 input_ids_layer (InputLayer)   [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids_layer (InputLay  [(None, 128)]       0           []                               
 er)                                                                                              
                                                                                              

In [ ]:
bert_classification_model_history = bert_classification_model.fit(
    [x_train_tf.input_ids, x_train_tf.token_type_ids, x_train_tf.attention_mask],
    y_train_tf,
    validation_data=([x_test_tf.input_ids, x_test_tf.token_type_ids, x_test_tf.attention_mask], y_test_tf),
    batch_size=32,
    epochs=2
) 

Epoch 1/2


In [ ]:
y_train_tf